<h2> Neural Network Music Generation

**INTRO *Evan

<h4> Recurrent Neural Networks vs. General Adversarial Networks

Below we present two deep learning methods for generating music: *Recurrent Neural Networks* (RNN) and *General Adversarial Networks* (GAN). We will explore the basic concepts of RNN and GNN to generate music. <br> <br>
But before we begin, we will import some helpul packages and tools to accomplish these tasks. 

In [ ]:
import os
import collections
import datetime
import glob
import numpy as np
import pathlib
import pandas as pd
from typing import Dict, List, Optional, Sequence, Tuple

#MIIDI file processing tools
import pretty_midi
import seaborn as sns
import fluidsynth

#Neural Network Tools
import tensorflow as tf
from keras.preprocessing import image
from keras.applications.xception import preprocess_input, decode_predictions
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from keras.utils.vis_utils import model_to_dot

#Plotting Tools
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
from keras.utils.vis_utils import plot_model

<h3> Recurrent Neural Network (RNN)

<h3> General Adversarial Neural Network (GAN)